# Importing modules

In [1]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression

import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

import hyperopt

# !pip install catboost
from catboost import CatBoostRegressor, Pool, metrics, cv

from jupyter_datatables import init_datatables_mode
init_datatables_mode() # visualise data tables

!jupyter nbextension enable --py widgetsnbextension

<JupyterRequire.display.SafeScript object>

<JupyterRequire.display.SafeScript object>

<JupyterRequire.display.SafeScript object>

<JupyterRequire.display.SafeScript object>

<JupyterRequire.display.SafeScript object>

<JupyterRequire.display.SafeScript object>

<JupyterRequire.display.SafeScript object>

<JupyterRequire.display.SafeScript object>

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: ok


In [17]:
# define MAPE score
def mape(actual, pred):
    return np.mean(np.abs((actual - pred) / actual)) * 100

# Data processing

In [2]:
# get the preferences data
data_pref_raw = pd.read_csv("SCI_and_preferences.csv", encoding="utf-8").iloc[:, 1:]
data_pref_raw = data_pref_raw.drop_duplicates(subset=data_pref_raw.columns[2:]).reset_index(drop=True)
data_pref_raw.scaled_sci = np.log(data_pref_raw.scaled_sci)

pref_features = list(data_pref_raw.columns[2:-1])
log_SCI = [data_pref_raw.columns[-1]]

In [3]:
# get the exogenous variables (containing extra regions and duplicates)
data_exog_raw = pd.read_csv("SCI_and_exogenous_variables.csv", encoding="utf-8").iloc[:, 1:]
data_exog_raw["log_distance"] = np.log(data_exog_raw.distance)

exog_features = [i for i in data_exog_raw.columns[2:] if i not in ["both_west_de", "both_east_de"]]

In [4]:
# combine the datasets omitting extra regions and duplicates
data_joint = pd.merge(data_pref_raw, data_exog_raw,  how="inner", left_on=["user_loc", "fr_loc"], right_on=["user_loc", "fr_loc"])

In [5]:
# create two datasets with equal number of rows, but different sets of predictors
data_pref = data_joint.loc[:, log_SCI + pref_features] # preferences
data_exog = data_joint.loc[:, log_SCI + exog_features] # exogenous variables

In [6]:
# set random seed for reproducibility
rng = np.random.RandomState(1)

In [7]:
#create a uniform placebo dataset
data_uniform_placebo = data_pref.copy()

for col in pref_features:
    data_uniform_placebo[col] = rng.uniform(size=data_pref.shape[0])

In [10]:
#create a normal placebo dataset
data_normal_placebo = data_pref.copy()

for col in pref_features:
    data_normal_placebo[col] = rng.normal(size=data_pref.shape[0])

# Building models for prediction of $log(SCI)$

## 1) $log(SCI)$ ~ preferences

In [21]:
X = data_pref.loc[:, pref_features]
y = data_pref.loc[:, log_SCI]

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=1)

In [10]:
# fit a boosting regressor with default parameters
boosting_reg = CatBoostRegressor(custom_metric=[metrics.MAPE(), metrics.R2()], logging_level='Silent', random_state=1)
boosting_reg.fit(X_train, y_train, plot=True);

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [21]:
print('train MAPE = {}%'.format(mape(y_train.scaled_sci, boosting_reg.predict(X_train))))
print('test MAPE = {}%'.format(mape(y_test.scaled_sci, boosting_reg.predict(X_test))))
print('train R_squared = {}'.format(r2_score(y_train.scaled_sci, boosting_reg.predict(X_train))))
print('test R_squared = {}'.format(r2_score(y_test.scaled_sci, boosting_reg.predict(X_test))))

train MAPE = 5.488378429259207%
test MAPE = 8.234840276270475%
train R_squared = 0.9033327337919508
test R_squared = 0.7746200710672916


In [20]:
# perform cross-validation to determine the optimal number of trees
cv_params = boosting_reg.get_params()

cv_data = cv(
    Pool(X_train, y_train),
    cv_params,
    iterations=3000,
    fold_count=5,
    plot=True
)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [32]:
# tune the hyperparameters
def hyperopt_objective(params):
    model = CatBoostRegressor(
        l2_leaf_reg=int(params['l2_leaf_reg']),
        learning_rate=params['learning_rate'],
        iterations=2000,
        eval_metric=metrics.MAPE(),
        random_seed=1,
        verbose=False,
        loss_function=metrics.RMSE(),
    )
    
    cv_data = cv(
        Pool(X_train, y_train),
        model.get_params()
    )
    
    best_MAPE = np.min(cv_data["test-MAPE-mean"])
    
    return best_MAPE

In [33]:
params_space = {
    'l2_leaf_reg': hyperopt.hp.qloguniform('l2_leaf_reg', 1, 4, 1),
    'learning_rate': hyperopt.hp.uniform('learning_rate', 1e-3, 5e-1),
}

trials = hyperopt.Trials()

best = hyperopt.fmin(
    hyperopt_objective,
    space=params_space,
    algo=hyperopt.tpe.suggest,
    max_evals=10,
    trials=trials,
    rstate=np.random.RandomState(1)
)

100%|██████████████████████████████████████████████| 10/10 [56:17<00:00, 337.75s/trial, best loss: 0.08398234676971289]


In [34]:
best

{'l2_leaf_reg': 21.0, 'learning_rate': 0.1205357630756511}

In [70]:
# fit again with tuned hyperparameters
boosting_reg = CatBoostRegressor(
    l2_leaf_reg=21.0, 
    learning_rate=0.1205357630756511,
    iterations=2000,
    custom_metric=[metrics.MAPE(), metrics.R2()], 
    logging_level='Silent', 
    random_state=1
)

boosting_reg.fit(X_train, y_train, plot=True);

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [49]:
print('train MAPE = {}%'.format(mape(y_train.scaled_sci, boosting_reg.predict(X_train))))
print('test MAPE = {}%'.format(mape(y_test.scaled_sci, boosting_reg.predict(X_test))))
print('train R_squared = {}'.format(r2_score(y_train.scaled_sci, boosting_reg.predict(X_train))))
print('test R_squared = {}'.format(r2_score(y_test.scaled_sci, boosting_reg.predict(X_test))))

train MAPE = 2.8370840853728128%
test MAPE = 7.614206389083779%
train R_squared = 0.9766553694965441
test R_squared = 0.8047177250592741


In [66]:
# feature importances
feature_importances = boosting_reg.get_feature_importance(Pool(X_test, y_test), type="PredictionValuesChange")
feature_names = X_train.columns
for score, name in sorted(zip(feature_importances, feature_names), reverse=True):
    print('{}: {}'.format(name, score))

child_hard_work: 7.101020731923989
Justifiable_Claiming_government_benefits_to_which_you_are_not_entitled: 6.479797912986653
It_is_childs_duty_to_take_care_of_ill_parent: 3.8450913687825867
child_independence: 3.6341811724453277
Religious_denomination: 3.386456030247135
Greater_respect_for_authority: 3.328786077063077
Willingness_to_fight_for_country: 2.886670036978651
How_often_do_you_attend_religious_services: 2.879246177429207
Trust_People_you_know_personally_(B): 2.184844936840127
Trust_Your_family_(B): 2.123131677547407
child_feeling_of_responsibility: 2.100468450223712
Justifiable_Homosexuality: 2.036665140018402
child_unselfishness: 1.6048683105712085
signing_a_petition: 1.4638818726873266
Pray_to_God_outside_of_religious_services_(EVS5): 1.2755955979102014
Neighbours_different_race: 1.2681105890415503
Believe_in_God: 1.2383028797821123
Trust_People_you_meet_for_the_first_time_(B): 1.2126899385551357
Most_people_can_be_trusted: 1.1985005068092565
Neighbours_Immigrants: 1.1874196

In [63]:
# feature importances (different method)
feature_importances = boosting_reg.get_feature_importance(Pool(X_test, y_test), type="LossFunctionChange")
feature_names = X_train.columns
for score, name in sorted(zip(feature_importances, feature_names), reverse=True):
    print('{}: {}'.format(name, score))

child_hard_work: 0.03248112861358465
Justifiable_Claiming_government_benefits_to_which_you_are_not_entitled: 0.03063368391308008
Pray_to_God_outside_of_religious_services_(EVS5): 0.029876204155073083
child_independence: 0.018746221765085336
Religious_denomination: 0.014543811007314389
Greater_respect_for_authority: 0.013974481939705408
It_is_childs_duty_to_take_care_of_ill_parent: 0.01209178929364818
child_feeling_of_responsibility: 0.012090522840423001
Trust_Your_family_(B): 0.01206215875182104
How_often_do_you_attend_religious_services: 0.01131774275752917
Willingness_to_fight_for_country: 0.010288545543908123
child_unselfishness: 0.00863315196709391
Trust_People_you_know_personally_(B): 0.0067593622810792064
Political_system_Having_a_strong_leader: 0.006425891269950967
Neighbours_Immigrants: 0.005728318374344621
signing_a_petition: 0.005603495999942476
Justifiable_Homosexuality: 0.005221207969781205
Democracy_People_obey_their_rulers: 0.004890843031280068
child_determination_perseve

In [9]:
lin_reg = LinearRegression()

lin_reg.fit(X_train, y_train);

In [12]:
print('train MAPE = {}%'.format(mape(y_train, lin_reg.predict(X_train))[0]))
print('test MAPE = {}%'.format(mape(y_test, lin_reg.predict(X_test))[0]))
print('train R_squared = {}'.format(r2_score(y_train, lin_reg.predict(X_train))))
print('test R_squared = {}'.format(r2_score(y_test, lin_reg.predict(X_test))))

train MAPE = 14.62379457214769%
test MAPE = 14.713679609575298%
train R_squared = 0.3685999719006382
test R_squared = 0.3760911994764822


In [27]:
# data for the gravity regression
X_train_gravity, X_test_gravity = X_train.copy(), X_test.copy()

for col in X_train_gravity.columns:
    X_train_gravity[col] = np.log1p(X_train_gravity[col])
    X_test_gravity[col] = np.log1p(X_test_gravity[col])

In [28]:
lin_reg = LinearRegression()

lin_reg.fit(X_train_gravity, y_train);

In [29]:
print('train MAPE = {}%'.format(mape(y_train, lin_reg.predict(X_train_gravity))[0]))
print('test MAPE = {}%'.format(mape(y_test, lin_reg.predict(X_test_gravity))[0]))
print('train R_squared = {}'.format(r2_score(y_train, lin_reg.predict(X_train_gravity))))
print('test R_squared = {}'.format(r2_score(y_test, lin_reg.predict(X_test_gravity))))

train MAPE = 14.776349756522574%
test MAPE = 14.823028327903314%
train R_squared = 0.3592687058159386
test R_squared = 0.3711478424487422


## 2) $log(SCI)$ ~ exogenous variables

In [71]:
X = data_exog.loc[:, exog_features]
y = data_exog.loc[:, log_SCI]

categorical_features_indices = np.where(X.dtypes != np.float)[0]

In [72]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=1)

In [27]:
# fit a boosting regressor with default parameters
boosting_reg = CatBoostRegressor(custom_metric=[metrics.MAPE(), metrics.R2()], logging_level='Silent', random_state=1)
boosting_reg.fit(X_train, y_train, cat_features=categorical_features_indices, plot=True);

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [28]:
print('train MAPE = {}%'.format(mape(y_train.scaled_sci, boosting_reg.predict(X_train))))
print('test MAPE = {}%'.format(mape(y_test.scaled_sci, boosting_reg.predict(X_test))))
print('train R_squared = {}'.format(r2_score(y_train.scaled_sci, boosting_reg.predict(X_train))))
print('test R_squared = {}'.format(r2_score(y_test.scaled_sci, boosting_reg.predict(X_test))))

train MAPE = 11.553017250876609%
test MAPE = 11.90309229015608%
train R_squared = 0.6597626418849509
test R_squared = 0.6533325518639037


In [29]:
# perform cross-validation to determine the optimal number of trees
cv_params = boosting_reg.get_params()

cv_data = cv(
    Pool(X_train, y_train, cat_features=categorical_features_indices),
    cv_params,
    iterations=3000,
    fold_count=5,
    plot=True
)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [39]:
# tune the hyperparameters
def hyperopt_objective(params):
    model = CatBoostRegressor(
        l2_leaf_reg=int(params['l2_leaf_reg']),
        learning_rate=params['learning_rate'],
        iterations=1000,
        eval_metric=metrics.MAPE(),
        random_seed=1,
        verbose=False,
        loss_function=metrics.RMSE(),
    )
    
    cv_data = cv(
        Pool(X_train, y_train, cat_features=categorical_features_indices),
        model.get_params()
    )
    
    best_MAPE = np.min(cv_data["test-MAPE-mean"])
    
    return best_MAPE

In [40]:
params_space = {
    'l2_leaf_reg': hyperopt.hp.qloguniform('l2_leaf_reg', 0, 4, 1),
    'learning_rate': hyperopt.hp.uniform('learning_rate', 1e-3, 5e-1),
}

trials = hyperopt.Trials()

best = hyperopt.fmin(
    hyperopt_objective,
    space=params_space,
    algo=hyperopt.tpe.suggest,
    max_evals=50,
    trials=trials,
    rstate=np.random.RandomState(1)
)

100%|███████████████████████████████████████████████| 50/50 [18:26<00:00, 22.13s/trial, best loss: 0.11892849987217069]


In [41]:
best

{'l2_leaf_reg': 1.0, 'learning_rate': 0.1316765532194309}

In [76]:
# fit again with tuned hyperparameters
boosting_reg = CatBoostRegressor(
    l2_leaf_reg=1.0, 
    learning_rate=0.1316765532194309,
    iterations=1000,
    custom_metric=[metrics.MAPE(), metrics.R2()], 
    logging_level='Silent', 
    random_state=1
)

boosting_reg.fit(X_train, y_train, cat_features=categorical_features_indices, plot=True);

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [77]:
print('train MAPE = {}%'.format(mape(y_train.scaled_sci, boosting_reg.predict(X_train))))
print('test MAPE = {}%'.format(mape(y_test.scaled_sci, boosting_reg.predict(X_test))))
print('train R_squared = {}'.format(r2_score(y_train.scaled_sci, boosting_reg.predict(X_train))))
print('test R_squared = {}'.format(r2_score(y_test.scaled_sci, boosting_reg.predict(X_test))))

train MAPE = 11.408734878153481%
test MAPE = 11.96630713837443%
train R_squared = 0.6670979141444966
test R_squared = 0.6476729461198691


In [78]:
# feature importances
feature_importances = boosting_reg.get_feature_importance(
    Pool(X_test, y_test, cat_features=categorical_features_indices), 
    type="PredictionValuesChange"
)

feature_names = X_train.columns
for score, name in sorted(zip(feature_importances, feature_names), reverse=True):
    print('{}: {}'.format(name, score))

country_1990_same: 31.45294389906891
distance: 10.49603259042357
user_eu_2003: 10.382614299959943
country_1960_same: 8.897264006859729
log_distance: 8.678221175740664
fr_eu_2003: 5.4762038820288685
same_main_language: 5.200581826180633
country_border: 4.60909550487868
both_austro_hung_empire: 3.9703749792242653
both_eastern_bloc: 3.5174951693348557
country_1900_same: 3.0874606950637524
both_uk_1900: 1.372726767872643
both_uk_1960: 0.9879573408583664
both_se_no_1900: 0.6740875208579684
country_1930_same: 0.5453269975544487
both_ru_1900: 0.28005858611898904
country_same: 0.11987740739247789
both_de_1900: 0.08882036256193843
both_soviet_union: 0.0751218368783983
both_czechoslovakia: 0.07450574364185673
both_yugoslavia: 0.011432137214298832
both_de_1930: 0.001797270284717889


In [79]:
# feature importances (different method)
feature_importances = boosting_reg.get_feature_importance(
    Pool(X_test, y_test, cat_features=categorical_features_indices), 
    type="LossFunctionChange"
)

feature_names = X_train.columns
for score, name in sorted(zip(feature_importances, feature_names), reverse=True):
    print('{}: {}'.format(name, score))

country_1990_same: 0.05440442254305833
log_distance: 0.027892461521972667
distance: 0.025896943335592892
country_1900_same: 0.02111293414789328
country_1960_same: 0.013858325973443941
both_austro_hung_empire: 0.013352062771386342
user_eu_2003: 0.011804364182419036
country_border: 0.007379134674863286
same_main_language: 0.00602970251856394
fr_eu_2003: 0.002275302873293583
both_se_no_1900: 0.0011920796928756028
both_eastern_bloc: 0.0009976138468870133
country_1930_same: 0.000986778139188793
both_de_1900: 0.00034434004526784356
both_uk_1900: 0.00026777290996926606
both_yugoslavia: 0.00014383046608834604
both_uk_1960: 8.068408111150394e-05
both_czechoslovakia: 5.507215137934285e-05
country_same: 3.653153111571328e-06
both_de_1930: -3.562309182858492e-06
both_ru_1900: -0.00014143428639157296
both_soviet_union: -0.00035538194496331776


In [44]:
lin_reg = LinearRegression()

lin_reg.fit(X_train, y_train)

LinearRegression()

In [45]:
print('train MAPE = {}%'.format(mape(y_train, lin_reg.predict(X_train))[0]))
print('test MAPE = {}%'.format(mape(y_test, lin_reg.predict(X_test))[0]))
print('train R_squared = {}'.format(r2_score(y_train, lin_reg.predict(X_train))))
print('test R_squared = {}'.format(r2_score(y_test, lin_reg.predict(X_test))))

train MAPE = 12.591634870545834%
test MAPE = 12.672519870148982%
train R_squared = 0.5981079185491187
test R_squared = 0.6107835826711983


# 3) $log(SCI)$ ~ preferences + exogenous variables

In [82]:
X = data_joint.loc[:, pref_features + exog_features]
y = data_joint.loc[:, log_SCI]

categorical_features_indices = np.where(X.dtypes != np.float)[0]

In [83]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=1)

In [84]:
# fit a boosting regressor with default parameters
boosting_reg = CatBoostRegressor(custom_metric=[metrics.MAPE(), metrics.R2()], logging_level='Silent', random_state=1)
boosting_reg.fit(X_train, y_train, cat_features=categorical_features_indices, plot=True);

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [85]:
print('train MAPE = {}%'.format(mape(y_train.scaled_sci, boosting_reg.predict(X_train))))
print('test MAPE = {}%'.format(mape(y_test.scaled_sci, boosting_reg.predict(X_test))))
print('train R_squared = {}'.format(r2_score(y_train.scaled_sci, boosting_reg.predict(X_train))))
print('test R_squared = {}'.format(r2_score(y_test.scaled_sci, boosting_reg.predict(X_test))))

train MAPE = 4.071533709526317%
test MAPE = 5.995802888988918%
train R_squared = 0.9548329725549245
test R_squared = 0.9030528197500531


In [86]:
# perform cross-validation to determine the optimal number of trees
cv_params = boosting_reg.get_params()

cv_data = cv(
    Pool(X_train, y_train, cat_features=categorical_features_indices),
    cv_params,
    iterations=3000,
    fold_count=5,
    plot=True
)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [96]:
# tune the hyperparameters
def hyperopt_objective(params):
    model = CatBoostRegressor(
        l2_leaf_reg=int(params['l2_leaf_reg']),
        learning_rate=params['learning_rate'],
        iterations=2500,
        eval_metric=metrics.MAPE(),
        random_seed=1,
        verbose=False,
        loss_function=metrics.RMSE(),
    )
    
    cv_data = cv(
        Pool(X_train, y_train, cat_features=categorical_features_indices),
        model.get_params()
    )
    
    best_MAPE = np.min(cv_data["test-MAPE-mean"])
    
    return best_MAPE

In [97]:
params_space = {
    'l2_leaf_reg': hyperopt.hp.qloguniform('l2_leaf_reg', 0, 4, 1),
    'learning_rate': hyperopt.hp.uniform('learning_rate', 1e-3, 5e-1),
}

trials = hyperopt.Trials()

best = hyperopt.fmin(
    hyperopt_objective,
    space=params_space,
    algo=hyperopt.tpe.suggest,
    max_evals=50,
    trials=trials,
    rstate=np.random.RandomState(1)
)

100%|███████████████████████████████████████████| 50/50 [5:56:48<00:00, 428.17s/trial, best loss: 0.059585526433243406]


In [98]:
best

{'l2_leaf_reg': 3.0, 'learning_rate': 0.06333624221419663}

In [99]:
# fit again with tuned hyperparameters
boosting_reg = CatBoostRegressor(
    l2_leaf_reg=3.0, 
    learning_rate=0.06333624221419663,
    iterations=2500,
    custom_metric=[metrics.MAPE(), metrics.R2()], 
    logging_level='Silent', 
    random_state=1
)

boosting_reg.fit(X_train, y_train, cat_features=categorical_features_indices, plot=True);

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [100]:
print('train MAPE = {}%'.format(mape(y_train.scaled_sci, boosting_reg.predict(X_train))))
print('test MAPE = {}%'.format(mape(y_test.scaled_sci, boosting_reg.predict(X_test))))
print('train R_squared = {}'.format(r2_score(y_train.scaled_sci, boosting_reg.predict(X_train))))
print('test R_squared = {}'.format(r2_score(y_test.scaled_sci, boosting_reg.predict(X_test))))

train MAPE = 2.2380851505775334%
test MAPE = 5.514195881102879%
train R_squared = 0.9865553766158637
test R_squared = 0.9156461542623184


In [92]:
# feature importances
feature_importances = boosting_reg.get_feature_importance(
    Pool(X_test, y_test, cat_features=categorical_features_indices), 
    type="PredictionValuesChange"
)

feature_names = X_train.columns
for score, name in sorted(zip(feature_importances, feature_names), reverse=True):
    print('{}: {}'.format(name, score))

country_1990_same: 20.849295147342197
country_1960_same: 8.44676688910983
distance: 3.7950692762946723
Justifiable_Claiming_government_benefits_to_which_you_are_not_entitled: 2.992372535008022
Justifiable_Homosexuality: 2.8537143768031985
log_distance: 2.6586100621313937
fr_eu_2003: 2.0306652859122085
both_austro_hung_empire: 1.6692139726137525
Political_system_Having_a_strong_leader: 1.5262737923779761
country_same: 1.4903684469111107
user_eu_2003: 1.3325061228372495
child_independence: 1.3239335392623777
country_border: 1.3076287443069265
Religious_denomination: 1.244459966194858
Trust_Your_family_(B): 1.2434670622937052
Trust_People_you_know_personally_(B): 1.1951427480766736
Neighbours_Homosexuals: 1.1934136345317448
It_is_childs_duty_to_take_care_of_ill_parent: 1.1167460672818956
Justifiable_Prostitution: 0.9621435570741517
Less_importance_placed_on_work: 0.947650209877854
child_hard_work: 0.9356455666821717
country_1900_same: 0.9206281664044281
Neighbours_Immigrants: 0.9058573555

In [93]:
# feature importances (different method)
feature_importances = boosting_reg.get_feature_importance(
    Pool(X_test, y_test, cat_features=categorical_features_indices), 
    type="LossFunctionChange"
)

feature_names = X_train.columns
for score, name in sorted(zip(feature_importances, feature_names), reverse=True):
    print('{}: {}'.format(name, score))

country_1990_same: 0.11191031933968204
distance: 0.08449466448817305
country_1960_same: 0.05360735111443288
log_distance: 0.043240763766623724
Justifiable_Claiming_government_benefits_to_which_you_are_not_entitled: 0.021801374927673534
Justifiable_Homosexuality: 0.016032983567858428
both_austro_hung_empire: 0.01571700862601355
Neighbours_Homosexuals: 0.010418900683716814
Political_system_Having_a_strong_leader: 0.009500341281180491
Pray_to_God_outside_of_religious_services_(EVS5): 0.008567724062215787
country_border: 0.008509100483506393
country_1900_same: 0.008500974048050336
Trust_Your_family_(B): 0.007114639666066014
Neighbours_Immigrants: 0.006985947381761726
Trust_People_you_know_personally_(B): 0.006423423445196974
child_independence: 0.006219388026534822
Religious_denomination: 0.00610884956161456
same_main_language: 0.005726538055209673
fr_eu_2003: 0.005587489475730512
both_se_no_1900: 0.005557469545233784
child_hard_work: 0.0052067251321459285
user_eu_2003: 0.00447186834663021

In [94]:
lin_reg = LinearRegression()

lin_reg.fit(X_train, y_train)

LinearRegression()

In [95]:
print('train MAPE = {}%'.format(mape(y_train, lin_reg.predict(X_train))[0]))
print('test MAPE = {}%'.format(mape(y_test, lin_reg.predict(X_test))[0]))
print('train R_squared = {}'.format(r2_score(y_train, lin_reg.predict(X_train))))
print('test R_squared = {}'.format(r2_score(y_test, lin_reg.predict(X_test))))

train MAPE = 9.636729448719155%
test MAPE = 9.756911901510495%
train R_squared = 0.7519636225258275
test R_squared = 0.7585671966113464


# 4) $log(SCI)$ ~ uniform placebo

In [23]:
X = data_uniform_placebo.loc[:, pref_features]
y = data_uniform_placebo.loc[:, log_SCI]

In [24]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=1)

In [25]:
# fit a boosting regressor with default parameters
boosting_reg = CatBoostRegressor(custom_metric=[metrics.MAPE(), metrics.R2()], logging_level='Silent', random_state=1)
boosting_reg.fit(X_train, y_train, plot=True);

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [26]:
print('train MAPE = {}%'.format(mape(y_train.scaled_sci, boosting_reg.predict(X_train))))
print('test MAPE = {}%'.format(mape(y_test.scaled_sci, boosting_reg.predict(X_test))))
print('train R_squared = {}'.format(r2_score(y_train.scaled_sci, boosting_reg.predict(X_train))))
print('test R_squared = {}'.format(r2_score(y_test.scaled_sci, boosting_reg.predict(X_test))))

train MAPE = 12.724787114820119%
test MAPE = 18.02596663983792%
train R_squared = 0.5071602982109926
test R_squared = -0.024999716440471387


In [27]:
lin_reg = LinearRegression()

lin_reg.fit(X_train, y_train);

In [28]:
print('train MAPE = {}%'.format(mape(y_train, lin_reg.predict(X_train))[0]))
print('test MAPE = {}%'.format(mape(y_test, lin_reg.predict(X_test))[0]))
print('train R_squared = {}'.format(r2_score(y_train, lin_reg.predict(X_train))))
print('test R_squared = {}'.format(r2_score(y_test, lin_reg.predict(X_test))))

train MAPE = 17.433482072684647%
test MAPE = 17.829415568085718%
train R_squared = 0.007202702224818003
test R_squared = -0.009151475108923668


# 4) $log(SCI)$ ~ normal placebo

In [13]:
X = data_uniform_placebo.loc[:, pref_features]
y = data_uniform_placebo.loc[:, log_SCI]

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=1)

In [15]:
# fit a boosting regressor with default parameters
boosting_reg = CatBoostRegressor(custom_metric=[metrics.MAPE(), metrics.R2()], logging_level='Silent', random_state=1)
boosting_reg.fit(X_train, y_train, plot=True);

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [18]:
print('train MAPE = {}%'.format(mape(y_train.scaled_sci, boosting_reg.predict(X_train))))
print('test MAPE = {}%'.format(mape(y_test.scaled_sci, boosting_reg.predict(X_test))))
print('train R_squared = {}'.format(r2_score(y_train.scaled_sci, boosting_reg.predict(X_train))))
print('test R_squared = {}'.format(r2_score(y_test.scaled_sci, boosting_reg.predict(X_test))))

train MAPE = 12.724787114820076%
test MAPE = 18.025966639837932%
train R_squared = 0.5071602982109926
test R_squared = -0.024999716440471387


In [19]:
lin_reg = LinearRegression()

lin_reg.fit(X_train, y_train);

In [20]:
print('train MAPE = {}%'.format(mape(y_train, lin_reg.predict(X_train))[0]))
print('test MAPE = {}%'.format(mape(y_test, lin_reg.predict(X_test))[0]))
print('train R_squared = {}'.format(r2_score(y_train, lin_reg.predict(X_train))))
print('test R_squared = {}'.format(r2_score(y_test, lin_reg.predict(X_test))))

train MAPE = 17.433482072684644%
test MAPE = 17.8294155680857%
train R_squared = 0.007202702224818003
test R_squared = -0.009151475108923668
